In [ ]:
import os
import random
import skimage.data
import skimage.transform
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Allow image embeding in notebook
%matplotlib inline

categories = {'warning':0, 'speedlimit':1, 'stop':2,
             0:'warning', 1:'speedlimt', 2:'stop'}

def load_data(data_dir, ann_dir):
    #returns a tuple of the relevant images and the relevant labels
    labels, images = [], []
    x1, x2, y1, y2 = 0, 0, 0, 0
    count = 0 #REMOVE WHEN MORE COMPUTING AVAILABLE
    with open(data_dir) as imset:
        for cur_im in imset:
            if count > 30: #REMOVE WHEN MORE COMPUTING AVAILABLE ##################
                break
            if cur_im.endswith("\n"):
                cur_im = cur_im[:-1] 
            with open(os.path.join(ann_dir, cur_im + ".txt")) as annotation:
                for anno in annotation:
                    label,x1,y1,x2,y2,clean = anno.split(',')
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    if "clean" in data_dir:
                        image = skimage.data.imread(os.path.join("Images", cur_im+".png"))
                    elif "ysq" in data_dir:
                        if os.path.exists(os.path.join("targeted_attack","stop-speedlimit-ysq",cur_im+".png")):
                            image = skimage.data.imread(os.path.join("targeted_attack","stop-speedlimit-ysq",cur_im+".png"))
                            label = 'speedlimit'
                        else:
                            image = skimage.data.imread(os.path.join("Images", cur_im+".png"))
                    labels.append(categories[label])
                    max_h, max_w = image.shape[0], image.shape[1]
                    image = skimage.util.crop(image,((y1, max_h - y2),(x1,max_w - x2),(0,0)), copy=False)
                    images.append(image)
                    count += 1 #REMOVE WHEN MORE COMPUTING AVAILABLE
        return images, labels

ROOT_PATH = ""
train_data_dir = os.path.join(ROOT_PATH, "ImageSets/test_ysq.txt")
test_data_dir = os.path.join(ROOT_PATH, "ImageSets/test_targ_ysg_backdoor.txt")
anno_dir = os.path.join(ROOT_PATH, "Annotations")

images, labels = load_data(train_data_dir, anno_dir)
print("Unique Labels: {0}\nTotal Images: {1}".format(len(set(labels)), len(images)))